## Q1 -

#### Solar (BEV) (i)

Here, we will calculate the KWh/m2/year data obtained by a solar PV cell at Phoenix and Boston. 
Using the "PV_hourly_output_modeling" fie used in the previous homework, we can extract the hourly GHI values (W/m2) at each of the two cities. 

We can then calculate the value in KWh/m2/year by summing over all the 8760 hours and finally diving by 1000 (to convert Wh/m2/year to KWh/m2/year)

In [1]:
import pandas as pd
import pvlib
from pvlib.location import Location
from pvlib.pvsystem import PVSystem, Array, FixedMount, SingleAxisTrackerMount
from pvlib.modelchain import ModelChain
from pvlib.iotools import get_psm3


year = 2022
api_key = 'jkAA6ShRTEOGlGziggu6aU84s2fB2l6U0J21Uis1'  # Replace with your actual API key
email = 'ag11023@nyu.edu'  # Replace with your actual email

#### Boston

In [2]:
#lat, lon = 33.4, -112.1   # Example: Phoenix, AZ
lat, lon = 42.3555, -71.0565  # Example: Boston, MA


data, meta = get_psm3(latitude=lat, longitude=lon, names=str(year), api_key=api_key, email=email, map_variables=True)

data.index = data.index.tz_convert('America/New_York')
#data.index = data.index.tz_convert('America/Phoenix')


location = Location(latitude=lat, longitude=lon, tz='America/New_York', altitude=meta['altitude'])

In [3]:
# Sum up the GHI column (units: W/m²)
ghi_sum = data["ghi"].sum()  # Wh/m² since each entry is hourly average in W/m² * 1 h

# Convert to kWh/m²/year
ghi_annual_kwh_per_m2_boston = ghi_sum / 1000

print(f"Annual GHI (Boston)= {ghi_annual_kwh_per_m2_boston:.2f} kWh/m²/year")

Annual GHI (Boston)= 1502.22 kWh/m²/year


#### Phoenix

In [4]:
latP, lonP = 33.4, -112.1   # Example: Phoenix, AZ
dataP, metaP = get_psm3(latitude=latP, longitude=lonP, names=str(year), api_key=api_key, email=email, map_variables=True)

dataP.index = dataP.index.tz_convert('America/Phoenix')


locationP = Location(latitude=latP, longitude=lonP, tz='America/Phoenix', altitude=metaP['altitude'])

In [5]:
# Sum up the GHI column (units: W/m²)
ghi_sum_phoenix = dataP["ghi"].sum()  # Wh/m² since each entry is hourly average in W/m² * 1 h

# Convert to kWh/m²/year
ghi_annual_kwh_per_m2_phoenix = ghi_sum_phoenix / 1000

print(f"Annual GHI (Phoenix) = {ghi_annual_kwh_per_m2_phoenix:.2f} kWh/m²/year")

Annual GHI (Phoenix) = 2148.79 kWh/m²/year


Now, we obtain the electricity which can be generated from the PV plant at each location

In [6]:
efficiency = 0.2
effective_conversion = 0.95 # derate the system by 5%

effective_boston = ghi_annual_kwh_per_m2_boston * efficiency * effective_conversion
effective_phoenix = ghi_annual_kwh_per_m2_phoenix * efficiency * effective_conversion

print(f"Effective Annual Energy Output (Boston) = {effective_boston:.2f} kWh/m²/year")
print(f"Effective Annual Energy Output (Phoenix) = {effective_phoenix:.2f} kWh/m²/year")


Effective Annual Energy Output (Boston) = 285.42 kWh/m²/year
Effective Annual Energy Output (Phoenix) = 408.27 kWh/m²/year


We now calculate the KWh requirement for a BEV per mile.

We can do this using the mpgge value

BEV rated mpgge (Miles per gallon of gasoline equivalent) given : 86

1 gallon of gasoline equivalent (GGE) = 32.880 KWh

86 miles per GGE means that the car can cover 86 miles per 33.7 KWh of energy consumption.
Hence, for 1 mile, a BEV car under the given specifications requires 33.7/86 = 0.391 KWh of energy.

Once we have this value. We can apply simple unitary method to calculate the number of miles a BEV can run using effective annual energy output of each city. Then, simply inversing the value obtained tells us the land area requirement per mile of BEV for each city.


In [7]:
gge = 32.880  # kWh/gallon
mpgge = 86
energy_per_mile = gge / mpgge  # kWh/mile

miles_travelled_using_1m2_area_boston = effective_boston / energy_per_mile
miles_travelled_using_1m2_area_phoenix = effective_phoenix / energy_per_mile

area_needed_for_1_mile_boston = 1 / miles_travelled_using_1m2_area_boston
area_needed_for_1_mile_phoenix = 1 / miles_travelled_using_1m2_area_phoenix

print(f"Area needed to travel 1 mile (Boston) = {area_needed_for_1_mile_boston:.4f} m²")
print(f"Area needed to travel 1 mile (Phoenix) = {area_needed_for_1_mile_phoenix:.4f} m²")

Area needed to travel 1 mile (Boston) = 0.0013 m²
Area needed to travel 1 mile (Phoenix) = 0.0009 m²


#### (ii) ICEV - Using Biomass

The calculation is similar to the previous case. 

LHV = 18.6 MJ/kg
Collection Rate = 3 kg/m2/year
Efficiency = 50%
ICEV mpgge = 19

1 gallon of gasoline equivalent (GGE) = 32.880 KWh

19 miles per GGE means that the car can cover 19 miles per 32.880 KWh of energy consumption.
Hence, for 1 mile, a BEV car under the given specifications requires 32.880/19 = 1.773 KWh of energy. (energy per mile)

Now, energy generation from biomass - 

LHV*Collection Rate = 55.8 MJ/m2/year

(1 KWh = 3.6 MJ)

55.8 MJ/m2/year = 15.1 KWh/m2/year

Electricity generation = 15.1*0.5 = 7.75 KWh/m2/year

Miles travelled using 1m2 area = Electricity Generation / energy per mile

	7.75/1.773 = 4.369 miles/m2

	Area required per mile travelled = 1/4.369 = 0.228 m2.




In [8]:

mpgge = 19
energy_per_mile = 33.7 / mpgge  # kWh/mile
lhv = 18.6 # MJ/kg
collection_rate = 3 # kg/m²/year
collection_rate_kwh = lhv * collection_rate * (1/3.6)  # kWh/m²/year
efficiency_bio = 0.50
effective_bio = collection_rate_kwh * efficiency_bio

print(f"Effective Annual Energy Output from Biofuel = {effective_bio:.2f} kWh/m²/year")

Effective Annual Energy Output from Biofuel = 7.75 kWh/m²/year


In [9]:
# Miles travelled using 1 m² area of biofuel
miles_travelled_using_1m2_area_bio = effective_bio / energy_per_mile
# Area needed to travel 1 mile using biofuel
area_needed_for_1_mile_bio = 1 / miles_travelled_using_1m2_area_bio

print(f"Area needed to travel 1 mile (Biofuel) = {area_needed_for_1_mile_bio:.4f} m²")

Area needed to travel 1 mile (Biofuel) = 0.2289 m²


### (iii) Hybrid - Synthetic Fuel 

This uses carbon as CO2 obtained via DAC, powered using ssolar power, and h2 created again using solar power.

#### CO2 Calculation - 

Given data - 

Gasoline density = 2.836 kg/gal
Efficiency (Gasoline) = 0.828

This gives us the carbon mass per gallon of gasoline -> density * carbon fraction = 2.836 * 0.828 = 2.350 kg/gal (of Carbon)

Hence, CO2 mass per gallon -> 2.350 * 44 /12 = 8.619 kg/gal (CO2)
This means that per gallon of gasoline consumed, we can make 8.619 kg of CO2.

Now, the given MPGGE value is 19, which means a requirement of 1/19 gallons per mile. 

using this information, we can calculate the CO2 requirement per mile = 8.619/19 = 0.453 kg CO2/mile

Now, DAC electricty requirement = 15.7 GJ/tonne CO2.
We need to extract exactly 0.453 kg CO2 as carbon efficiency of synthetic fuel process is given as 100%
So, for 0.453 kg, the DAC electricity requirement is -> 0.453 * 15.7 = 7.114 MJ/mile (which should come from solar)

#### H2 calculation

Given the MPGGE value of 19, and the LHV of Gasoline as 118.371 MJ/gallon (converted from Btu).

We can compute the energy per mile required -> 6.230 MJ/mile.
Now, given an efficiency of 55% of synthetic fuel production process, the H2 feed LHV value can be computed -> 6.230/0.55 = 11.327 MJ/mile

Also, given is an efficiency of 50% (elctricty to H2 energy). Hence, solar energy required = 11.327/0.5 = 22.654 MJ/mile.

#### Solar requirement -> 

22.654 (for H2) + 7.114 (for DAC) = 29.769 MJ/mile = 8.269 KWh/mile

Now, from part (i) of this problem, we know that - 

Effective Annual Energy Output (Boston) = 285.42 kWh/m²/year
Effective Annual Energy Output (Phoenix) = 408.27 kWh/m²/year

Hence, for Boston

8.269/285.42 = 0.028 m2/mile

And for Phoenix 

8.269/408.27 = 0.020 m2/mile







In [10]:
## CO2 Emissions Calculations

gasoline_density = 2.84  # kg/gallon
gasoline_carbon_content = 0.828  # kg C/kg gasoline
carbon_mass_per_gallon = gasoline_density * gasoline_carbon_content  # kg C/gallon
Co2_per_gallon = carbon_mass_per_gallon * (44/12)  # kg CO2/gallon
mpgge = 19
co2_per_mile = Co2_per_gallon / mpgge  # kg CO2/mile

DAC_requirement_per_tonne = 15.7 # GJ/tonne CO2 = MJ/kg CO2
DAC_requirement = 0.4536 * DAC_requirement_per_tonne # requirement in MJ/kg CO2 based on the amount of CO2 required for synthetic fuel process

In [11]:
## H2 from electrolysis

gasoline_lhv = 118.371 # MJ/Co2_per_gallon
mpgge = 19
energy_mile = gasoline_lhv / mpgge  # MJ/mile
energy_required_for_synthetic_process = energy_mile/0.55 # MJ/mile
solar_output_required = energy_required_for_synthetic_process/0.5 # MJ/mile


In [12]:
## Solar requirement

total_solar_output_required = DAC_requirement + solar_output_required  # MJ/mile
total_solar_output_required_kwh = total_solar_output_required / 3.6  # kWh/mile

effective_boston = 285.42 ## KWh/m2/year - Calculated earlier
effective_phoenix = 408.27 ## KWh/m2/year - Calculated earlier

area_needed_for_1_mile_boston_solar = total_solar_output_required_kwh / effective_boston
area_needed_for_1_mile_phoenix_solar = total_solar_output_required_kwh / effective_phoenix

print(f"Area needed to travel 1 mile (Solar Boston) = {area_needed_for_1_mile_boston_solar:.4f} m²")
print(f"Area needed to travel 1 mile (Solar Phoenix) = {area_needed_for_1_mile_phoenix_solar:.4f} m²")

Area needed to travel 1 mile (Solar Boston) = 0.0290 m²
Area needed to travel 1 mile (Solar Phoenix) = 0.0203 m²


Running the above calculations, we can see the land requirement for a solar powered BEV car is an order of magnitude less than the synthetic fuel powered car. Even worse if the land requirement using biomass, which is an order of magnitude higher than the synthetic fuel requirement. 

This emphasizes on the effectiveness of solar power in terms of energy requirement per area. For this question, note that the land requirement order is BEV < Biomass ICEV < Synthetic ICEV.

The land calculation for Synthetic ICEV used solar as a source for production of H2 and CO2 for liquid fuel formation. Hence, in terms of land requirement it was merely an inefficient solar powered method.
